# Series Missing Value Methods

In the previous chapter, we covered the most common attributes and statistical methods for pandas Series objects. In this chapter, we cover several methods that handle missing values within a Series. The official documentation has a [separate section in its Series API][1] just for these methods that handle missing values. Let's begin by reading in the movie dataset and selecting the `duration` series, which contains the length of each movie in minutes.

[1]: https://pandas.pydata.org/pandas-docs/stable/reference/series.html#missing-data-handling

In [ ]:
import pandas as pd
movie = pd.read_csv('../data/movie.csv', index_col='title')
duration = movie['duration']
duration.head()

## Methods for handling missing values

pandas provides the following methods to handle missing values:

* `isna` - Returns a Series of booleans based on whether each value is missing or not
* `notna` - Exact opposite of `isna`
* `dropna` - Drops the missing values from the Series
* `fillna` - Fills missing values in a variety of ways
* `interpolate` - Fills missing values with statistical interpolation

As a reminder, boolean and integer columns do not allow missing values, so these methods will have not be useful for Series with those data types.

## The `isna` method

The `isna` method tests whether each value in the Series is missing or not. It returns a new boolean Series where `True` corresponds with missing values. Non-missing values return as `False`. Let's call the `isna` method on our `duration` Series. One of the first five values is missing.

In [ ]:
duration.isna().head()

### Counting the number of missing values

pandas does not have a single method that counts the number of missing values in a Series. We are forced to do a little work to find the result. Calling the `sum` method after the `isna` method will count the number of missing values as `True`/`False` evaluate a 1/0.

In [ ]:
duration.isna().sum()

Alternatively, you can use the `count` method to count the number of non-missing values and subtract it from the length of the Series.

In [ ]:
len(duration) - duration.count()

### Finding the percentage of missing values
To find the percentage of missing values in a Series, chain the `mean` method to the `isna` method.

In [ ]:
duration.isna().mean()

### Alternate calculation
The last calculation might be confusing. We could have been more explicit and calculated the percentage of missing values by dividing the number missing by the total size of the Series as done below.

In [ ]:
total = len(duration)
num_missing = total - duration.count()
num_missing / total

### Why does taking the mean of the boolean Series work?

The mean is defined as the sum of all values divided by the total number of values. In the case of a boolean Series, its sum is just the number of `True` values and in this specific example is equal to the number of missing values.

### The `notna` method

The `notna` method works analogously to the `isna` method but returns `True` for non-missing values. Let's verify that it returns the exact opposite result as `isna`.

In [ ]:
duration.notna().head()

Let's count the number of non-missing values by summing up the previous result.

In [ ]:
duration.notna().sum()

This is the exact calculation that the `count` method performs, so the above is unnecessary.

In [ ]:
duration.count()

## Dropping missing values with `dropna`

The `dropna` method returns a new Series without any missing values. In the `duration` Series, the fifth value was missing. After calling `dropna`, it's no longer there.

In [ ]:
duration2 = duration.dropna()
duration2.head()

Above, we calculated that there were 15 missing values in the Series. Let's verify that the length of the new Series has decreased by this amount.

In [ ]:
len(duration2)

In [ ]:
len(duration)

## Filling missing values with the `fillna` method

There are a number of ways that have been developed to fill missing values. Some of these are quite complex and involve machine learning. pandas only provides a couple simple choices with the `fillna` method. Let's read a dataset with data on a person's weight over time. There are several missing values here.

In [ ]:
weight_loss = pd.read_csv('../data/weight_loss.csv')
weight_loss

In [ ]:
weight = weight_loss['weight']
weight

There are a few options with the `fillna` method. Passing it a single number replaces each missing value with that number. Let's fill in all missing values with 200.

In [ ]:
weight.fillna(200)

A common strategy involves filling missing values with either the mean or the median. Let's calculate the median, assign it to a variable name, and then call the `fillna` method. Here, the median is 201.

In [ ]:
median = weight.median()
weight.fillna(median)

### Filling with the previous or next non-missing value

The `fillna` method provides an option to use either the previous or next non-missing value. To fill the missing value with the most recent previous non-missing value, set the `method` parameter to 'ffill', which stands for 'forward fill'. Let's forward fill our `weight` Series. Notice that the first value remains missing as there is no non-missing value that comes before it.

In [ ]:
weight.fillna(method='ffill')

Similarly, we can fill the missing values with the next non-missing value by setting the `method` parameter to 'bfill', which stands for 'backward fill'. This time the last value remains missing as no non-missing value follows it.

In [ ]:
weight.fillna(method='bfill')

### Limit the number of missing values filled

By default, all missing values will be filled when the `method` parameter is set. You can limit the number of consecutive missing values filled with the `limit` parameter by setting it equal to an integer. Here, we only allow the very next missing value to be filled.

In [ ]:
weight.fillna(method='ffill', limit=1)

## Filling missing values with `interpolate`

Interpolation is a more complex method for filling in missing values that gives you more control as to how the values are filled. There are many different methods for interpolation that all involve statistical calculations. In pandas, we use the `interpolate` method and choose the type of interpolation with the `method` parameter. By default, pandas does linear interpolation.

In [ ]:
weight.interpolate(method='linear')

Linear interpolation is the simplest method and computes missing values such that they are modeled using a straight line. With the `weight` Series, 205 is the 2nd value and 201 the fifth. There are two missing values between them. We compute the slope of the line between these two points as $\frac{201 - 205}{5 - 2} = -\frac{4}{3}$.

The missing values are filled in with the equation $205 - \frac{4}{3}x$ where $x$ is the number of values from the last non-missing value. This process repeats for the next interval of missing values with a new slope calculated. Interestingly, pandas fills any missing values that appear after the last non-missing value with that last non-missing value. You can prevent this by setting the `limit_area` parameter to 'inside'.

In [ ]:
weight.interpolate(method='linear', limit_area='inside')

### Many more interpolations methods

There are many more interpolation methods such as quadratic, cubic, spline, and others. All of these methods are processed by the **scipy** library's interpolate module. You'll need to [navigate to the scipy documentation][1] to read the exact details of how the methods work. Here, we interpolate using the quadratic method.

[1]: https://docs.scipy.org/doc/scipy/reference/interpolate.html

In [ ]:
weight.interpolate('quadratic')

## Graphing interpolation methods

Comparing different interpolation methods can be aided with a graph. Here, we compare three different methods:

* `fillna` with forward filling
* linear interpolation
* quadratic interpolation

The plotting commands will be explained in the **Visualizations** part of the course.

In [ ]:
%matplotlib inline
weight.fillna(method='ffill').plot(kind='line', label='ffill', legend=True, figsize=(12, 6))
weight.interpolate('linear').plot(kind='line', label='linear', legend=True)
weight.interpolate('quadratic').plot(kind='line', label='quadratic', legend=True);

## Interpolation methods use the index

All of the interpolation methods (except linear) use the index values during their calculation. Below, we create a copy of our data and change the index to be numbers other than the sequence of integers beginning at 0. Notice, how the interpolated values are different from above. 

In [ ]:
w2 = weight.copy()
w2.index = [0, 1, 5, 6, 8, 12, 22, 27, 30, 35]
w2.interpolate('quadratic')

## Exercises

Execute the following cell, to use the actor 1 and actor 2 Facebook like Series for the first few exercises.

In [ ]:
actor1_fb = movie['actor1_fb']
actor2_fb = movie['actor2_fb']
actor1_fb.head()

In [ ]:
actor2_fb.head()

### Exercise 1

<span  style="color:green; font-size:16px">What percentage of actor 1 Facebook likes are missing?</span>

### Exercise 2

<span  style="color:green; font-size:16px">Use the `notna` method to find the number of non-missing values in the actor 1 Facebook like column. Verify this number is the same as the `count` method.</span>

### Exercise 3

<span  style="color:green; font-size:16px">Fill the missing values of `actor1_fb` with the maximum of `actor2_fb`. Save this result to variable `actor1_fb_full`</span>

### Exercise 4

<span  style="color:green; font-size:16px">Verify the results of Exercise 3 by selecting just the values of `actor1_fb_full` that were filled by `actor2_fb`.</span>

### Exercise 5

<span  style="color:green; font-size:16px">Use the `duration` Series and test whether each movie is greater than 100. Assign the resulting Series to `filt`. Then test whether the `duration` Series is less than or equal to 100 and assign it to `filt2. Call the `sum` method on both of these new Series and add their results together. Why doesn't this result equal the total length of the Series? Shouldn't a value be either greater than 100 or less than or equal to 100?</span>

### Exercise 6

<span  style="color:green; font-size:16px">How many missing values are there in the `year` column.</span>

### Exercise 7

<span  style="color:green; font-size:16px">Select the language column as a Series and assign it to a variable with the same name. Create a variable `filt` that determines whether a language is missing. Create a new Series that fills in all missing languages with 'English' and assign it to the variable `language2`. Output both `language` and `language2` for the movies that originally had missing values.</span>

### Exercise 7

<span  style="color:green; font-size:16px">Repeat exercise 7 without first assigning the language column to a variable. Reference it by using *just the brackets*. Still make a variable `language2` to verify th</span>

### Exercise 8

<span  style="color:green; font-size:16px">Select the `gross` column, and drop all missing values from it. Confirm that the new length of the resulting Series is correct.</span>

### Exercise 9

<span  style="color:green; font-size:16px">Read in `girl_height.csv` as a DataFrame and output all of the data. The average height for each age is found in the `height` column. Assign the `height_na` Series to a variable. Notice that all ages from 2 through 12 are missing, but all other ages have the same value as the `height` column. Use the `interpolate` method to fill in the missing values with method 'linear', 'quadratic', and 'cubic'. Save each interpolated Series to a variable with the same name as its method.</span>

### Exercise 10

<span  style="color:green; font-size:16px">Uncomment and run the commands below to plot each interpolated Series. Which one provides the best estimate for height?</span>

In [ ]:
# %matplotlib inline
# girl_height['height'].plot(figsize=(12, 6), label='actual', legend=True)
# linear.plot(label='linear', legend=True)
# quadratic.plot(label='quadratic', legend=True)
# cubic.plot(label='cubic', legend=True)